In [3]:
### Merge and Map Chess Openings

# Import libraries
import pandas as pd
from io import StringIO


In [4]:
#tsv data
a=pd.read_csv('a.tsv', sep='\t')
b=pd.read_csv('b.tsv', sep='\t')
c=pd.read_csv('c.tsv', sep='\t')
d=pd.read_csv('d.tsv', sep='\t')
e=pd.read_csv('e.tsv', sep='\t')

tsv_data= a,b,c,d,e

merged_df = pd.concat(tsv_data, ignore_index=True)

In [18]:
output_file_path = 'merged_openings.csv'
merged_df.to_csv(output_file_path, index=False)

In [5]:
merged_df.head()

,eco,name,pgn
0,A00,Amar Opening,1. Nh3
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6
4,A00,Anderssen's Opening,1. a3


In [6]:


opening_moves = merged_df

def split_pgn_to_columns(df, pgn_column='pgn'):
    max_plies = 0
    split_data = []
    plies_list = []

    # Split each PGN into individual plies and find the maximum number of plies
    for pgn in df[pgn_column]:
        moves = [move for move in pgn.split() if not move.endswith('.')]
        split_data.append(moves)
        plies_list.append(" ".join(moves))  # Join moves into a single string for 'plies' column
        max_plies = max(max_plies, len(moves))
    
    # Add 'plies' column to the DataFrame
    df['plies'] = plies_list

    # Create new columns for each ply
    columns = {f'Move_ply_{i+1}': [] for i in range(max_plies)}

    # Assign moves to corresponding columns
    for moves in split_data:
        for i in range(max_plies):
            if i < len(moves):
                columns[f'Move_ply_{i+1}'].append(moves[i])
            else:
                columns[f'Move_ply_{i+1}'].append(None)
    
    # Add new columns to the DataFrame
    for col_name, col_values in columns.items():
        df[col_name] = col_values

    return df

# Apply the function
opening_moves = split_pgn_to_columns(opening_moves)

print(opening_moves.head())


   eco                                     name  \
0  A00                             Amar Opening   
1  A00               Amar Opening: Paris Gambit   
2  A00  Amar Opening: Paris Gambit, Gent Gambit   
3  A00                         Amsterdam Attack   
4  A00                      Anderssen's Opening   

                                                 pgn  \
0                                             1. Nh3   
1                           1. Nh3 d5 2. g3 e5 3. f4   
2  1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...   
3             1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6   
4                                              1. a3   

                                          plies Move_ply_1 Move_ply_2  \
0                                           Nh3        Nh3       None   
1                               Nh3 d5 g3 e5 f4        Nh3         d5   
2  Nh3 d5 g3 e5 f4 Bxh3 Bxh3 exf4 O-O fxg3 hxg3        Nh3         d5   
3                    e3 e5 c4 d6 Nc3 Nc6 b3 Nf6         e3      

In [7]:
print(opening_moves['plies'])

0                                                     Nh3
1                                         Nh3 d5 g3 e5 f4
2            Nh3 d5 g3 e5 f4 Bxh3 Bxh3 exf4 O-O fxg3 hxg3
3                              e3 e5 c4 d6 Nc3 Nc6 b3 Nf6
4                                                      a3
                              ...                        
3463    d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
3464    d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
3465    d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
3466    d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
3467    d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
Name: plies, Length: 3468, dtype: object


In [8]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.opening_name = None

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, moves, opening_name):
        node = self.root
        for move in moves:
            if move not in node.children:
                node.children[move] = TrieNode()
            node = node.children[move]
        node.opening_name = opening_name
    
    def search(self, moves):
        node = self.root
        last_opening_name = None
        for move in moves:
            if move in node.children:
                node = node.children[move]
                if node.opening_name:
                    last_opening_name = node.opening_name
            else:
                break
        return last_opening_name

In [9]:
trie = Trie()
for index, row in opening_moves.iterrows():
    opening = row['name']
    plies = row['plies'].split()
    trie.insert(plies, opening)

In [10]:
game_data = pd.read_csv('C:\\Users\\clancyam\\OneDrive - University of South Australia\\2024\\SP4\\Capstone Project\\Code\\Mapping Openings\\chessdata.csv')

C:\Users\clancyam\AppData\Local\Temp\ipykernel_12972\1502522186.py:1: DtypeWarning: Columns (188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408) have mixed types. Specify dtype option on import or set low_memory=False.
  game_data = pd.read_csv('C:\\Users\\clancyam\\OneDrive - University of South Australia\\2024\\SP4\\Capstone Project\\Code\\Mapping Openings\\chessdata.csv')


In [11]:
game_data = pd.DataFrame(game_data)

In [13]:
game_data.columns

Index(['Index', 'Date', 'ECO', 'Opening', 'Result', 'Termination',
       'TimeControl', 'UTCDate', 'UTCTime', 'Move_ply_1',
       ...
       'Clock_ply_192', 'Clock_ply_193', 'Clock_ply_194', 'Clock_ply_195',
       'Clock_ply_196', 'Clock_ply_197', 'Clock_ply_198', 'Clock_ply_199',
       'Clock_ply_200', 'Category'],
      dtype='object', length=410)

In [16]:
# Select the first 36 move columns as the largest opening definition consists of 18 moves
move_columns = [f'Move_ply_{i+1}' for i in range(36)]
selected_moves_df = game_data[move_columns + ['Opening']].copy()

# Concatenate moves into a single sequence for each game
def create_move_sequence(row):
    moves = row.dropna().tolist()  # Drop NaN values and convert to list
    return ' '.join(moves)

selected_moves_df['move_sequence'] = selected_moves_df.apply(create_move_sequence, axis=1)

# Function to get the opening name from a sequence of moves
def get_opening_name(moves):
    return trie.search(moves.split())

# Apply the function to get the opening names
selected_moves_df['opening_name'] = selected_moves_df['move_sequence'].apply(get_opening_name)

# Compare 'opening_name' with 'name'
selected_moves_df['match'] = selected_moves_df['opening_name'] == selected_moves_df['Opening']

# Print the DataFrame with the move sequences, identified openings, and match result
print(selected_moves_df[['move_sequence', 'opening_name', 'Opening', 'match']])

# Calculate the percentage of matches
match_percentage = (selected_moves_df['match'].sum() / len(selected_moves_df)) * 100
print(f"Percentage of matches: {match_percentage:.2f}%")

                                            move_sequence  \
0       d4 d5 Nc3 c6 e4 h5 exd5 cxd5 Qf3 Bg4 Qf4 e6 Bb...   
1       e4 e5 Nf3 Nc6 Bc4 d6 Nc3 h6 d4 exd4 Nxd4 Nf6 N...   
2       e4 e5 Nf3 d6 c3 Be7 d4 exd4 cxd4 Nf6 Bd3 h6 O-...   
3       e4 g6 d4 h6 Nc3 Bg7 Nge2 e6 f3 Ne7 Be3 O-O Qd2...   
4       e4 c5 Nf3 Nc6 d4 cxd4 Nxd4 e5 Nb5 a6 Nd6+ Bxd6...   
...                                                   ...   
199995  e4 Nc6 d4 e5 d5 Nce7 c4 Ng6 Be2 Bb4+ Bd2 Bxd2+...   
199996  e4 e5 d4 d5 exd5 Qxd5 Nf3 Bg4 Be2 e4 Nc3 Qa5 N...   
199997  d4 d5 c4 dxc4 Nc3 e6 e3 Nf6 Bxc4 c6 a4 a5 b3 B...   
199998  d4 d6 e4 g6 e5 Bg7 exd6 exd6 d5 Nf6 c3 O-O Nd2...   
199999  d4 e6 Bf4 c5 dxc5 Bxc5 Nc3 d5 Nf3 Nf6 Bg5 O-O ...   

                                             opening_name  \
0                   Queen's Pawn Game: Chigorin Variation   
1                             Italian Game: Paris Defense   
2                                        Philidor Defense   
3                      

In [17]:
output_file_path = 'opening_comparison_results.csv'
selected_moves_df.to_csv(output_file_path, index=False)

print(f"Results exported to {output_file_path}")

Results exported to opening_comparison_results.csv


,eco,name,pgn,Move_ply_1,Move_ply_2,Move_ply_3,Move_ply_4,Move_ply_5,Move_ply_6,Move_ply_7,...,Move_ply_28,Move_ply_29,Move_ply_30,Move_ply_31,Move_ply_32,Move_ply_33,Move_ply_34,Move_ply_35,Move_ply_36,plies
0,A00,Amar Opening,1. Nh3,Nh3,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Nh3
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4,Nh3,d5,g3,e5,f4,None,None,...,None,None,None,None,None,None,None,None,None,Nh3 d5 g3 e5 f4
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...,Nh3,d5,g3,e5,f4,Bxh3,Bxh3,...,None,None,None,None,None,None,None,None,None,Nh3 d5 g3 e5 f4 Bxh3 Bxh3 exf4 O-O fxg3 hxg3
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6,e3,e5,c4,d6,Nc3,Nc6,b3,...,None,None,None,None,None,None,None,None,None,e3 e5 c4 d6 Nc3 Nc6 b3 Nf6
4,A00,Anderssen's Opening,1. a3,a3,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,a3
